# Square Bells Sweep Analysis

Minimal workflow to inspect one sweep directory and make two figures:

1. `a0`, `a1`, `b1` for one indexed run `(i, j)`
2. normalized strip profiles of `b1(x)` at fixed `y`, across selected `gamma_mc`

Field convention in this notebook: `field[x_index, y_index]`.


In [ ]:
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Matplotlib styling (Seaborn is used only for colormaps)
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman", "Times New Roman", "DejaVu Serif"],
    "font.size": 13,
    "axes.titlesize": 14,
    "axes.labelsize": 13,
    "legend.fontsize": 11,
})

# ---- User config ----
SWEEP_DATA_DIR = Path("projects/square_bells_ucsb/results/square_bells_ucsb_sweep_p=1.0_2026-02-08_010941/data")
I_IDX = 0  # index in gamma_mr grid
J_IDX = 0  # index in gamma_mc grid


In [ ]:
def parse_params_from_filename(filename: str) -> dict:
    stem = filename[:-3] if filename.endswith(".h5") else filename
    keys = ["bias", "gamma_mc", "gamma_mr", "p_scatter", "index_global"]
    params = {}
    for key in keys:
        marker = f"{key}="
        if marker not in stem:
            continue
        tail = stem.split(marker, 1)[1]
        value = tail.split("_", 1)[0]
        try:
            params[key] = float(value)
        except ValueError:
            params[key] = value
    return params


def load_observables(path: Path):
    # transpose once so notebook consistently uses field[x_index, y_index]
    with h5py.File(path, "r") as f:
        return {
            "a0": f["a0"][:].T,
            "a1": f["a1"][:].T,
            "b1": f["b1"][:].T,
            "x": f["x"][:],
            "y": f["y"][:],
            "meta": dict(f.attrs),
        }


In [ ]:
if not SWEEP_DATA_DIR.exists():
    raise FileNotFoundError(f"Sweep data directory not found: {SWEEP_DATA_DIR}")

runs = []
for path in sorted(SWEEP_DATA_DIR.glob("*.h5")):
    runs.append((parse_params_from_filename(path.name), path))

if len(runs) == 0:
    raise RuntimeError(f"No .h5 files found in {SWEEP_DATA_DIR}")

gamma_mr_vals = sorted({p["gamma_mr"] for p, _ in runs if "gamma_mr" in p})
gamma_mc_vals = sorted({p["gamma_mc"] for p, _ in runs if "gamma_mc" in p})

x_to_i = {v: i for i, v in enumerate(gamma_mr_vals)}
y_to_j = {v: j for j, v in enumerate(gamma_mc_vals)}
by_ij = {}
for params, path in runs:
    if "gamma_mr" in params and "gamma_mc" in params:
        by_ij[(x_to_i[params["gamma_mr"]], y_to_j[params["gamma_mc"]])] = (params, path)

print(f"sweep: {SWEEP_DATA_DIR}")
print(f"runs: {len(runs)}")
print(f"gamma_mr values: {len(gamma_mr_vals)}")
print(f"gamma_mc values: {len(gamma_mc_vals)}")
print(f"indexed entries: {len(by_ij)}")

if (I_IDX, J_IDX) not in by_ij:
    available = sorted(by_ij.keys())
    raise KeyError(f"No run at (i={I_IDX}, j={J_IDX}). Example available index: {available[0] if available else None}")

params, run_path = by_ij[(I_IDX, J_IDX)]
data = load_observables(run_path)

gamma_mr = params["gamma_mr"]
gamma_mc = params["gamma_mc"]

print(f"selected (i,j)=({I_IDX},{J_IDX})")
print(f"gamma_mr={gamma_mr}, gamma_mc={gamma_mc}")
print(f"file: {run_path.name}")
print(f"field shape: {data['a0'].shape}")


## Figure 1: `a0`, `a1`, `b1` for one run


In [ ]:
a0 = np.asarray(data["a0"], dtype=float)
a1 = np.asarray(data["a1"], dtype=float)
b1 = np.asarray(data["b1"], dtype=float)
x = np.asarray(data["x"], dtype=float)
y = np.asarray(data["y"], dtype=float)

cmap_a0 = sns.color_palette("mako", as_cmap=True)
cmap_div = sns.color_palette("icefire", as_cmap=True)

a1_max = np.nanmax(np.abs(a1))
b1_max = np.nanmax(np.abs(b1))

X, Y = np.meshgrid(x, y, indexing="ij")

fig, axes = plt.subplots(1, 3, figsize=(16.5, 4.8), constrained_layout=True)

m0 = axes[0].pcolormesh(X, Y, a0, cmap=cmap_a0, shading="auto")
axes[0].set_title(r"$a_0$")
axes[0].set_xlabel(r"$x$")
axes[0].set_ylabel(r"$y$")
fig.colorbar(m0, ax=axes[0], label=r"$a_0$")

m1 = axes[1].pcolormesh(X, Y, a1, cmap=cmap_div, vmin=-a1_max, vmax=a1_max, shading="auto")
axes[1].set_title(r"$a_1$")
axes[1].set_xlabel(r"$x$")
axes[1].set_ylabel(r"$y$")
fig.colorbar(m1, ax=axes[1], label=r"$a_1$")

m2 = axes[2].pcolormesh(X, Y, b1, cmap=cmap_div, vmin=-b1_max, vmax=b1_max, shading="auto")
axes[2].set_title(r"$b_1$")
axes[2].set_xlabel(r"$x$")
axes[2].set_ylabel(r"$y$")
fig.colorbar(m2, ax=axes[2], label=r"$b_1$")

fig.suptitle(
    rf"Run $(i,j)=({I_IDX},{J_IDX})$: $\gamma_{{mr}}={gamma_mr:.3g}$, $\gamma_{{mc}}={gamma_mc:.3g}$",
    fontsize=15,
)
plt.show()


## Figure 2: normalized strip profiles of `b1(x)`

At fixed `gamma_mr` (`I_IDX`), this plots selected `gamma_mc` curves at fixed `y = Y_CUT`.
Each curve is normalized by `I_y(y_*) = \int b_1(x, y_*) dx`.


In [ ]:
# Cut configuration
Y_CUT = 0.40
N_GAMMA_MC_CURVES = 8

i_index = I_IDX
if i_index >= len(gamma_mr_vals):
    raise IndexError(f"I_IDX={i_index} out of bounds for gamma_mr grid of size {len(gamma_mr_vals)}")

j_available = sorted(j for (i, j) in by_ij.keys() if i == i_index)
if len(j_available) == 0:
    raise RuntimeError(f"No runs found for i={i_index}")

n_pick = min(N_GAMMA_MC_CURVES, len(j_available))
pick_positions = np.linspace(0, len(j_available) - 1, n_pick).round().astype(int)
j_picks = [j_available[k] for k in pick_positions]

y_ref = np.asarray(data["y"], dtype=float)
y_index = int(np.argmin(np.abs(y_ref - Y_CUT)))
y_used = float(y_ref[y_index])

gamma_mc_sel = np.array([by_ij[(i_index, j)][0]["gamma_mc"] for j in j_picks], dtype=float)
gamma_pos = gamma_mc_sel[gamma_mc_sel > 0]
if gamma_pos.size == 0:
    raise RuntimeError("All selected gamma_mc values are nonpositive; log color scale is undefined.")
gamma_min = float(np.min(gamma_pos))
gamma_max = float(np.max(gamma_pos))
if np.isclose(gamma_min, gamma_max):
    gamma_max = gamma_min * 10.0

cmap_line = sns.color_palette("flare", as_cmap=True)
norm = plt.matplotlib.colors.LogNorm(vmin=gamma_min, vmax=gamma_max)

fig, ax = plt.subplots(figsize=(6.4, 4.2), constrained_layout=True)
plotted = 0

for j in j_picks:
    params_j, path_j = by_ij[(i_index, j)]
    d = load_observables(path_j)

    x = np.asarray(d["x"], dtype=float)
    b1 = np.asarray(d["b1"], dtype=float)

    # fixed-y strip profile along x
    b1_x = b1[:, y_index]

    finite_x = np.isfinite(x)
    if np.count_nonzero(finite_x) < 2:
        continue

    x_line = x[finite_x]
    b1_line = b1_x[finite_x]

    finite_b1 = np.isfinite(b1_line)
    if np.count_nonzero(finite_b1) < 2:
        continue

    I_y = float(np.trapezoid(b1_line[finite_b1], x_line[finite_b1]))
    if np.isclose(I_y, 0.0):
        continue

    gamma_mc = float(params_j["gamma_mc"])
    if gamma_mc <= 0:
        continue

    y_plot = b1_line / I_y

    # reverse layering: smaller gamma_mc on top
    z = norm(gamma_mc)
    ax.plot(x_line, y_plot, lw=2.0, color=cmap_line(norm(gamma_mc)), zorder=10.0 - 9.0 * z)
    plotted += 1

if plotted == 0:
    raise RuntimeError("No valid line cuts plotted. Try a different Y_CUT or check data quality for this i-index.")

sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap_line)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, pad=0.02)
cbar.set_label(r"$\gamma_{mc}$")

ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$b_1(x, y_*) / I_y(y_*)$")
ax.set_title(rf"Normalized $b_1(x)$ at fixed $\gamma_{{mr}}={gamma_mr_vals[i_index]:.3g}$ and $y_*\approx {y_used:.3g}$")
ax.grid(False)
plt.show()
